# Chapter 2: Our First Model

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

## Setting up DataLoaders

We'll use the built-in dataset of `torchvision.datasets.ImageFolder` to quickly set up some dataloaders of downloaded cat and fish images. 

`check_image`  is a quick little function that is passed to the `is_valid_file` parameter in the ImageFolder and will do a sanity check to make sure PIL can actually open the file. We're going to use this in lieu of cleaning up the downloaded dataset.


In [2]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

Set up the transforms for every image:

* Resize to 64x64
* Convert to tensor
* Normalize using ImageNet mean & std


In [3]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225] )
    ])



In [4]:
train_data_path = "../train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)

In [5]:
val_data_path = "../val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)

In [6]:
test_data_path = "../test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image) 

In [7]:
batch_size=64

In [8]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size) 

## Our First Model, SimpleNet

SimpleNet is a very simple combination of three Linear layers and ReLu activations between them. Note that as we don't do a `softmax()` in our `forward()`, we will need to make sure we do it in our training function during the validation phase.

In [9]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50,2)
    
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [10]:
simplenet = SimpleNet()

## Create an optimizer

Here, we're just using Adam as our optimizer with a learning rate of 0.001.

In [11]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

## Copy the model to GPU

Copy the model to the GPU if available.

In [12]:
# Test MPS availability
if torch.backends.mps.is_available():
    device = torch.device("mps") 
else:
    device = torch.device("cpu")

print(device)
simplenet.to(device)

mps


SimpleNet(
  (fc1): Linear(in_features=12288, out_features=84, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

## Training 

Trains the model, copying batches to the GPU if required, calculating losses, optimizing the network and perform validation for each epoch.

In [25]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [14]:
train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,val_data_loader, epochs=5, device=device)

Epoch: 1, Training Loss: 2.37, Validation Loss: 5.14, accuracy = 0.31
Epoch: 2, Training Loss: 2.29, Validation Loss: 0.94, accuracy = 0.72
Epoch: 3, Training Loss: 0.54, Validation Loss: 1.58, accuracy = 0.54
Epoch: 4, Training Loss: 0.66, Validation Loss: 0.90, accuracy = 0.64
Epoch: 5, Training Loss: 0.35, Validation Loss: 1.18, accuracy = 0.57


## Making predictions

Labels are in alphanumeric order, so `cat` will be 0, `fish` will be 1. We'll need to transform the image and also make sure that the resulting tensor is copied to the appropriate device before applying our model to it.

In [15]:
labels = ['cat','fish']

img = Image.open("../val/fish/100_1422.JPG") 
img = img_transforms(img).to(device)
img = torch.unsqueeze(img, 0)

simplenet.eval()
prediction = F.softmax(simplenet(img), dim=1)
prediction = prediction.argmax()
print(labels[prediction]) 

fish


## Saving Models

We can either save the entire model using `save` or just the parameters using `state_dict`. Using the latter is normally preferable, as it allows you to reuse parameters even if the model's structure changes (or apply parameters from one model to another).

In [16]:
torch.save(simplenet, "/tmp/simplenet") 
simplenet = torch.load("/tmp/simplenet")    


In [17]:
torch.save(simplenet.state_dict(), "/tmp/simplenet")    
simplenet = SimpleNet()
simplenet_state_dict = torch.load("/tmp/simplenet")
simplenet.load_state_dict(simplenet_state_dict)   

<All keys matched successfully>

## Tests with Simplenet

Before you read the next chapter, experiment with the SimpleNet architecture we created here. 
- Adjust the number of parameters in the Linear layers, and maybe add another layer or two. 
- Have a look at the various activation functions available in PyTorch and swap out ReLU for something else. 
- See what happens to training if you adjust the learning rate or switch out the optimizer from Adam to another option (perhaps try vanilla SGD). 
- Maybe alter the batch size and the initial size of the image as it gets turned into a 1D tensor at the start of the forward pass.

In [34]:
# Training loop
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):

    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            # Init gradients to 0
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            # Compute loss
            loss = loss_fn(output, targets)
            # Gradient descent
            loss.backward()
            # Update weights
            optimizer.step()
            # Add loss
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))


# Change Simplenet class
class SimpleNetUpdate(nn.Module):

    def __init__(self):
        super(SimpleNetUpdate, self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 10)
        self.fc4 = nn.Linear(10, 2)
    
    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.leaky_relu(self.fc3(x))
        x = self.fc4(x)
        return x
    

# Instantiate a network
simplenet_update = SimpleNetUpdate()
# Pass network to GPU
simplenet_update.to("mps")
# Set optimizer
optimizer = optim.Adam(simplenet_update.parameters(), lr=0.0001)

# Training loop
train(simplenet_update, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=20, device="mps")


Epoch: 1, Training Loss: 0.78, Validation Loss: 0.58, accuracy = 0.82
Epoch: 2, Training Loss: 0.66, Validation Loss: 0.63, accuracy = 0.69
Epoch: 3, Training Loss: 0.55, Validation Loss: 0.69, accuracy = 0.62
Epoch: 4, Training Loss: 0.51, Validation Loss: 0.71, accuracy = 0.64
Epoch: 5, Training Loss: 0.48, Validation Loss: 0.70, accuracy = 0.65
Epoch: 6, Training Loss: 0.45, Validation Loss: 0.69, accuracy = 0.66
Epoch: 7, Training Loss: 0.42, Validation Loss: 0.67, accuracy = 0.66
Epoch: 8, Training Loss: 0.39, Validation Loss: 0.67, accuracy = 0.66
Epoch: 9, Training Loss: 0.38, Validation Loss: 0.65, accuracy = 0.69
Epoch: 10, Training Loss: 0.35, Validation Loss: 0.66, accuracy = 0.69
Epoch: 11, Training Loss: 0.34, Validation Loss: 0.65, accuracy = 0.69
Epoch: 12, Training Loss: 0.31, Validation Loss: 0.66, accuracy = 0.72
Epoch: 13, Training Loss: 0.30, Validation Loss: 0.65, accuracy = 0.71
Epoch: 14, Training Loss: 0.27, Validation Loss: 0.67, accuracy = 0.73
Epoch: 15, Trai